In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

_join = os.path.join

### Summary:

The purpose of this jupyter notebook is to create utilities for origin and destination pairs segmented by time period (AM, MD, PM, EA and EV) and purpose (Work, University, School, Escort, Shopping, EatOut, OthMaint, Social, OthDiscr, WorkBased). It carries out the following steps

- Read the landuse data to extract population, total employment, area and area type by taz. We determine the population and employment per square mile to assign the TNC wait time for each TAZs. 

- Read the data tab in the 'TripModeChoice.xlsx' file that lists all the matrix cores in the skims that are used in the utility equations. Read all the skims and store the skims in memory. 

- Assign PM CROWD and IWAIT of PNR Transit Walk skim to AM CROWD and IWAIT of PNR Transit Walk. 

- Iterate over time period and purpose to read the coefficients from 'params.properties' file and 'TripModeChoice.xlsx' file. calculate the utilities and save the results as OMX files.  

The final output from these scripts generates utilities for each mode for each OD pair and saves them as separate OMX files by different time period and purpose combinations.  

##### Path to model folders

In [2]:
# model_folders = ['W:\TM2_2050Baseline_R2_Run4', 'W:\TM2_2050R39_R2_Run4', 'W:\TM2_2050R41_R2_Run4', 'TM2_2050R40_R2_Run2']
# model_dir = 'W:\TM2_2050R40_R2_Run2' #update this to run for other model folders
# model_folders = ['W:\TM2_2050STR39_R2_Run2', 'W:\TM2_2050STR40_R2_Run1', 'W:\TM2_2050STR41_R2_Run2']

# model_dir = 'W:\TM2_2050R40_R2_Run2_VY'
# model_dir = 'D:\TM2_2050R40_R2_Run2_Conv'
# model_dir = 'Z:\TM2_2050_BL_R2_Run6_COVID_Run3'
#model_dir = 'Z:\TM2_2050_R39_R2_Run6_Cnvrg'
model_dir = 'D:\TM2_2050R41_R2_Run4_Conv'

model_year = 2050

##### Purpose and time period definitions

In [3]:
purpose = ['Work', 'University', 'School', 'Escort', 'Shopping', 'EatOut', 
           'OthMaint', 'Social', 'OthDiscr', 'WorkBased']

time_period = {1:'EA',2:'AM',3:'MD',4:'PM',5:'EV'} #1 for EA, 2 for AM, 3 for MD, 4 for PM and 5 for EV

In [4]:
# store all the utilities in this folder
preprocess_dir = _join(model_dir, '_pre_process', 'util_rev')

# create the folder if it doesn't exist
Path(preprocess_dir).mkdir(parents=True, exist_ok=True)

#### Load TAZs and create Area Types

In [5]:
taz = pd.read_csv(_join('tazData_' + str(model_year) + '.csv'))
taz['popEmpSqMile'] = (taz['TOTPOP'] + taz['TOTEMP']) / (taz['TOTACRE'] * 0.0015625)

In [6]:
taz_atype = taz[['ZONE', 'AREATYPE']] 
# Integer, 0=regional core, 1=central business district, 2=urban business, 3=urban, 4=suburban, 5=rural
# 1=San Francisco; 2=San Mateo; 3=Santa Clara; 4=Alameda; 5=Contra Costa; 6=Solano; 7= Napa; 8=Sonoma; 9=Marin

#### Calculate the taxi wait time for each origin zone

In [7]:
%%time
taz_pop = taz[['ZONE', 'popEmpSqMile']]

taz_pop['density_group'] = pd.cut(
    taz_pop['popEmpSqMile'],
    bins=[-1, 500, 2000, 5000, 15000, 9999999999],
    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
    ordered=False
)

taz_pop['density_group_values'] = 0
taz_pop.loc[taz_pop['density_group'] == 'Very Low', 'density_group_values'] = 10.3
taz_pop.loc[taz_pop['density_group'] == 'Low', 'density_group_values'] = 8.5
taz_pop.loc[taz_pop['density_group'] == 'Medium', 'density_group_values'] = 8.4
taz_pop.loc[taz_pop['density_group'] == 'High', 'density_group_values'] = 6.3
taz_pop.loc[taz_pop['density_group'] == 'Very High', 'density_group_values'] = 3.0

taz_pop = taz_pop.sort_values('ZONE')
taxi_wait_time = np.repeat(taz_pop['density_group_values'].values, len(taz_pop)).reshape(len(taz_pop), len(taz_pop))

Wall time: 64.3 ms


### Load all the data from Skims 

In [ ]:
%%time
# The data tab of the UEC file lists all the matrix cores and location an matrix files of skims
# 1 for EA, 2 for AM, 3 for MD, 4 for PM and 5 for EV

# extract the file names, matrix cores 
matrix_df = pd.read_excel(_join(r"TripModeChoice.xlsx"), sheet_name='data')
matrix_df = matrix_df.iloc[9:]
matrix_df.columns = ['no', 'token', 'format', 'file','matrix', 'group', 'index']
#matrix_df[1:5]

# pre-processing
matrix_df['matrix_files'] = matrix_df['file'].str.replace('skims/', '')
matrix_df['path'] = 'skims'

# Iterate over the DataFrame rows
for _, row in matrix_df.iterrows():
    variable_name = row['token']
    file_path = row['path']
    filename = row['matrix_files']
    matrix_cr = row['matrix']
    
    # Extract the variable name and index (if present)
    if '[' in variable_name:
        name_start = variable_name.index('[')
        name_end = variable_name.index(']')
        index = int(variable_name[name_start+1:name_end])
        variable_name = variable_name[:name_start]
    else:
        index=None
    
    # Read the file using numpy.load() and assign it to the variable with the specified index
    file = omx.open_file(_join(model_dir, file_path, filename))
    file_contents = np.array(file[matrix_cr])
    print(variable_name,index, _join(model_dir, file_path, filename), file_contents.sum(), file_contents.min(), file_contents.max())
    if '[' in row['token']:
        if variable_name in locals() and isinstance(locals()[variable_name], np.ndarray):
            arr = locals()[variable_name]
            if index >= len(arr):
                # Resize the array if the index is out of bounds
                new_arr = np.resize(arr, index + 1)
                new_arr[index] = file_contents
                locals()[variable_name] = new_arr
            else:
                arr[index] = file_contents
        else:
            arr = np.empty(index + 1, dtype=object)
            arr[index] = file_contents
            locals()[variable_name] = arr
    else:
        locals()[variable_name] = file_contents

DISTWALK None D:\TM2_2050R41_R2_Run4_Conv\skims\nonmotskm.omx 10796411595951.248 0.03142102696417673 1000000.0
DISTBIKE None D:\TM2_2050R41_R2_Run4_Conv\skims\nonmotskm.omx 8910510616075.47 0.018911417199612486 1000000.0
SOV_TIME 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 13965421000.0 0.11402574 1000000.0
SOV_DIST 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 13847429000.0 0.035375483 1000000.0
SOV_BTOLL 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 800990500.0 0.0 472.0
SOV_VTOLL 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 740808640.0 0.0 334.74524
HOV2_TIME 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 13958099000.0 0.11402574 1000000.0
HOV2_DIST 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 13843934000.0 0.035375483 1000000.0
HOV2_BTOLL 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 923095600.0 0.0 901.0
HOV2_VTOLL 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 759671040.0 0.0 321.3217
HOV3_TIME 1 D:\TM2_2050R41_R2_Run4_Conv\skims\hwyskmEA.omx 13955171000

WLK_TRN_WLK_FAR 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 5094786600.0 0.0 1631.2822
WLK_TRN_WLK_WAUX 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 12782970000.0 0.0 58221.082
WLK_TRN_WLK_IWAIT 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 8186385400.0 0.0 14628.0
WLK_TRN_WLK_XWAIT 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 8775963000.0 0.0 17065.998
WLK_TRN_WLK_BOARDS 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 24093606.0 0.0 8.139927
WLK_TRN_WLK_WACC 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 8670756000.0 0.0 1499.888
WLK_TRN_WLK_WEGR 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_WLK_TRN_WLK.omx 8865372000.0 0.0 1499.888
WLK_TRN_WLK_WAIT 3 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmMD_WLK_TRN_WLK.omx 16943889000.0 0.0 19609.525
WLK_TRN_WLK_TOTIVT 3 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmMD_WLK_TRN_WLK.omx 75725730000.0 0.0 38670.043
WLK_TRN_WLK_CROWD 3 D:\TM2_2050R41_R2_Run4_Conv\skims\

PNR_TRN_WLK_IVT_LRT 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 2193383700.0 0.0 8186.5854
PNR_TRN_WLK_IVT_FRY 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 4045573400.0 0.0 13936.273
PNR_TRN_WLK_IVT_HVY 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 28968944000.0 0.0 14111.841
PNR_TRN_WLK_IVT_COM 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 14749994000.0 0.0 17598.87
PNR_TRN_WLK_FAR 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 6261895700.0 0.0 1849.976
PNR_TRN_WLK_DTIM 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx -2.7226601e+26 -4.354166e+22 16980.04
PNR_TRN_WLK_DDIST 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 9664754000.0 0.0 12915.443
PNR_TRN_WLK_WAUX 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 7593250000.0 0.0 13644.645
PNR_TRN_WLK_IWAIT 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_PNR_TRN_WLK.omx 825772400.0 0.0 2407.0
PNR_TRN_WLK_XWAIT 2 D:\TM2_2050R41_

KNR_TRN_WLK_DDIST 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 11320025000.0 0.0 13760.056
KNR_TRN_WLK_WAUX 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 10989276000.0 0.0 65167.96
KNR_TRN_WLK_IWAIT 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 7561988000.0 0.0 4000.0
KNR_TRN_WLK_XWAIT 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 2773644800.0 0.0 6500.0
KNR_TRN_WLK_BOARDS 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 14919834.0 0.0 5.25
KNR_TRN_WLK_WACC 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 0.0 0.0 0.0
KNR_TRN_WLK_WEGR 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_KNR_TRN_WLK.omx 9662364000.0 0.0 1498.9987
KNR_TRN_WLK_WAIT 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_KNR_TRN_WLK.omx 11185029000.0 0.0 14628.0
KNR_TRN_WLK_TOTIVT 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_KNR_TRN_WLK.omx 55421223000.0 0.0 35832.527
KNR_TRN_WLK_CROWD 2 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmAM_KNR_TRN_WLK.

KNR_TRN_WLK_WEGR 5 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEV_KNR_TRN_WLK.omx 9449327000.0 0.0 1498.9985
WLK_TRN_PNR_WAIT 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_WLK_TRN_PNR.omx 12545556000.0 0.0 9672.995
WLK_TRN_PNR_TOTIVT 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_WLK_TRN_PNR.omx 50220880000.0 0.0 25672.201
WLK_TRN_PNR_CROWD 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_WLK_TRN_PNR.omx 244187460.0 0.0 893.49384
WLK_TRN_PNR_IVT_LOC 1 D:\TM2_2050R41_R2_Run4_Conv\skims\trnskmEA_WLK_TRN_PNR.omx 9279562000.0 0.0 12447.019


In [9]:
# change the 1000000.0 values in DISTWALK and DISTBIKE to 0
DISTWALK = np.where(DISTWALK == 1000000.0, 0, DISTWALK)
DISTBIKE = np.where(DISTBIKE == 1000000.0, 0, DISTBIKE)

# change any negative values to 0
PNR_TRN_WLK_DTIM[1] =  np.where(PNR_TRN_WLK_DTIM[1] < 0, 0, PNR_TRN_WLK_DTIM[1])
PNR_TRN_WLK_DTIM[2] =  np.where(PNR_TRN_WLK_DTIM[2] < 0, 0, PNR_TRN_WLK_DTIM[2])
PNR_TRN_WLK_DTIM[3] =  np.where(PNR_TRN_WLK_DTIM[3] < 0, 0, PNR_TRN_WLK_DTIM[3])
PNR_TRN_WLK_DTIM[4] =  np.where(PNR_TRN_WLK_DTIM[4] < 0, 0, PNR_TRN_WLK_DTIM[4])
PNR_TRN_WLK_DTIM[5] =  np.where(PNR_TRN_WLK_DTIM[5] < 0, 0, PNR_TRN_WLK_DTIM[5])

KNR_TRN_WLK_DTIM[1] =  np.where(KNR_TRN_WLK_DTIM[1] < 0, 0, KNR_TRN_WLK_DTIM[1])
KNR_TRN_WLK_DTIM[2] =  np.where(KNR_TRN_WLK_DTIM[2] < 0, 0, KNR_TRN_WLK_DTIM[2])
KNR_TRN_WLK_DTIM[3] =  np.where(KNR_TRN_WLK_DTIM[3] < 0, 0, KNR_TRN_WLK_DTIM[3])
KNR_TRN_WLK_DTIM[4] =  np.where(KNR_TRN_WLK_DTIM[4] < 0, 0, KNR_TRN_WLK_DTIM[4])
KNR_TRN_WLK_DTIM[5] =  np.where(KNR_TRN_WLK_DTIM[5] < 0, 0, KNR_TRN_WLK_DTIM[5])

WLK_TRN_PNR_DTIM[1] =  np.where(WLK_TRN_PNR_DTIM[1] < 0, 0, WLK_TRN_PNR_DTIM[1])
WLK_TRN_PNR_DTIM[2] =  np.where(WLK_TRN_PNR_DTIM[2] < 0, 0, WLK_TRN_PNR_DTIM[2])
WLK_TRN_PNR_DTIM[3] =  np.where(WLK_TRN_PNR_DTIM[3] < 0, 0, WLK_TRN_PNR_DTIM[3])
WLK_TRN_PNR_DTIM[4] =  np.where(WLK_TRN_PNR_DTIM[4] < 0, 0, WLK_TRN_PNR_DTIM[4])
WLK_TRN_PNR_DTIM[5] =  np.where(WLK_TRN_PNR_DTIM[5] < 0, 0, WLK_TRN_PNR_DTIM[5])

WLK_TRN_KNR_DTIM[1] =  np.where(WLK_TRN_KNR_DTIM[1] < 0, 0, WLK_TRN_KNR_DTIM[1])
WLK_TRN_KNR_DTIM[2] =  np.where(WLK_TRN_KNR_DTIM[2] < 0, 0, WLK_TRN_KNR_DTIM[2])
WLK_TRN_KNR_DTIM[3] =  np.where(WLK_TRN_KNR_DTIM[3] < 0, 0, WLK_TRN_KNR_DTIM[3])
WLK_TRN_KNR_DTIM[4] =  np.where(WLK_TRN_KNR_DTIM[4] < 0, 0, WLK_TRN_KNR_DTIM[4])
WLK_TRN_KNR_DTIM[5] =  np.where(WLK_TRN_KNR_DTIM[5] < 0, 0, WLK_TRN_KNR_DTIM[5])

In [10]:
PNR_TRN_WLK_DTIM[2]

array([[   0.     ,  779.08704,  779.08704, ...,  779.08704,  779.08704,
         779.08704],
       [ 821.83545,    0.     ,  821.83545, ...,  821.83545,  821.83545,
         821.83545],
       [1640.5724 , 1655.8319 ,    0.     , ..., 1592.5833 , 1592.5833 ,
        1592.5833 ],
       ...,
       [3429.484  , 3990.5095 , 2779.1914 , ...,    0.     , 3030.8975 ,
        2784.6318 ],
       [3515.3137 , 4076.3386 , 2865.021  , ..., 3404.139  ,    0.     ,
        2870.4612 ],
       [3547.0078 , 4102.4707 , 2906.8748 , ..., 3438.0464 , 3438.0464 ,
           0.     ]], dtype=float32)

In [11]:
# use the PM peak crowding and iwait variables, transpose them, and use them for the AM.
PNR_TRN_WLK_CROWD[2] = PNR_TRN_WLK_CROWD[4].T
PNR_TRN_WLK_IWAIT[2] = PNR_TRN_WLK_IWAIT[4].T

In [12]:
# ct ramp has params.properties which has certain parameter values used in the utility equations. 
# Following function extracts these values.

def extract_property_values(file_path, variables):
    property_values = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith('#'):
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip()
                if key in variables:
                    property_values[key] = value
    return property_values

In [13]:
%%time
for purp in purpose:
    print(purp)
    # read the purpose tab from the UEC file. 
    uec_purp_columns = ['No', 'Token', 'Description', 'Filter','Formula for variable', 
               'Index','Alt1', 'Alt2', 'Alt3', 'Alt4', 'Alt5', 'Alt6', 'Alt7', 'Alt8', 'Alt9']
    
    uec_purp = pd.read_excel(_join("TripModeChoice.xlsx"), sheet_name=purp)
    uec_purp = uec_purp.iloc[2:]
    uec_purp.columns = uec_purp_columns # assign column names
    
    # Removing NAs
    uec_purp_params_prop = uec_purp.loc[~uec_purp['Token'].isna()]
    # extract the parameters that have % in in their names, clean up-remove % and replace . with _
    uec_purp_params_prop = uec_purp_params_prop.loc[(uec_purp_params_prop['Formula for variable'].str.contains('%')==True)]
    uec_purp_params_prop['Formula for variable'] = uec_purp_params_prop['Formula for variable'].str.replace('%', '') 
    uec_purp_params_prop['Formula for variable'] = uec_purp_params_prop['Formula for variable'].str.replace(".", "_")
    # read parameters file
    file_path = _join('params.properties')
    # extract list of parameters
    prop_variables = list(uec_purp_params_prop['Formula for variable'])
    prop_variables_tokens = list(uec_purp_params_prop['Token'])
    prop_variables = [x.replace('_', '.') for x in prop_variables]

    values = extract_property_values(file_path, prop_variables)
    
    # Create a dictionary to store the extracted values
    extracted_values = {}

    # Assign the extracted values to the dictionary
    for variable, value in values.items():
        extracted_values[variable] = value

    # Print the values from the extracted_values dictionary 
    for variable, value in extracted_values.items():
        #print(f'{variable}: {value}')
        exec(f'{variable.replace(".", "_")} = {value}')
    
    
    # Assign the values to tokens
    # example costInitialTaxi = %taxi.baseFare%
    for _, row in uec_purp_params_prop.iterrows():
        variable_name = row['Token']
        expression = row['Formula for variable']

        # Evaluate the expression and store the result in the local environment
        try:
            # Evaluate the expression and store the result in the local environment
            if expression in locals() and isinstance(locals()[expression], np.ndarray):
                value = locals()[expression]
            else:
                value = eval(expression)

            exec(f'{variable_name} = value')
            #print(f"Variable '{variable_name}' is defined.")
        except NameError:
            #print(f"Variable '{variable_name}' is not defined.")
            continue

    
    uec_purp_params = uec_purp.loc[~uec_purp['Formula for variable'].isna()]
    uec_purp_params = uec_purp_params.loc[~uec_purp_params['Token'].isna()]
    uec_purp_params = uec_purp_params.loc[~(uec_purp_params['Formula for variable'].str.contains('if')==True)]
    uec_purp_params = uec_purp_params.loc[~(uec_purp_params['Formula for variable'].str.contains('%')==True)]

    uec_purp_params['Formula for variable'] = uec_purp_params['Formula for variable'].astype(str)
    uec_purp_params['Formula for variable'] = uec_purp_params['Formula for variable'].str.replace('@', '')

    key_column = 'Token'
    value_column = 'Formula for variable'

    # Create dictionary from selected columns
    data_dict = {}

    for _, row in uec_purp_params.iterrows():
        key = row[key_column]
        value = row[value_column]

        # Handle values that are strings
        if isinstance(value, str):
            try:
                value = int(value)
                data_dict[key] = value
            except ValueError:
                try:
                    value = float(value)
                    data_dict[key] = value
                except ValueError:
                    pass

    #get all the parameters
    variables = data_dict

    for _, row in uec_purp_params.iterrows():
        variable_name = row['Token']
        expression = row['Formula for variable']

        # Evaluate the expression and store the result in the local environment
        try:
            # Evaluate the expression and store the result in the local environment
            if expression in locals() and isinstance(locals()[expression], np.ndarray):
                value = locals()[expression]
            else:
                value = eval(expression)

            exec(f'{variable_name} = value')
        except NameError:
            #print(f"Variable '{variable_name}' is not defined.")
            continue
    
    #break
    
    int_zone = 3332
    da_util = np.empty((5, int_zone, int_zone))
    sr2_util = np.empty((5, int_zone, int_zone))
    sr3_util = np.empty((5, int_zone, int_zone))
    wlk_util =  np.empty((5, int_zone, int_zone))
    bike_util = np.empty((5, int_zone, int_zone))
    wlk_trn_wlk_util = np.empty((5, int_zone, int_zone))
    wlk_trn_pnr_util = np.empty((5, int_zone, int_zone))
    pnr_trn_wlk_util = np.empty((5, int_zone, int_zone))
    wlk_trn_knr_util = np.empty((5, int_zone, int_zone))
    knr_trn_wlk_util = np.empty((5, int_zone, int_zone))
    taxi_util = np.empty((5, int_zone, int_zone))
    
    for tripPeriod in time_period:

        util = omx.open_file(_join(preprocess_dir, f'util_{tripPeriod}_{purp}.omx'),'w')

        # Drive alone utility
        util['DA'] = np.where(SOV_TIME[tripPeriod][:int_zone, :int_zone]>0, c_ivt*SOV_TIME[tripPeriod][:int_zone, :int_zone], 0)
        print(tripPeriod, purp, 'DA', " ", np.array(util['DA']).min(), np.array(util['DA']).max())

        #Shared ride 2 utility
        util['SR2'] = np.where(HOV2_TIME[tripPeriod][:int_zone, :int_zone]>0, c_ivt*HOV2_TIME[tripPeriod][:int_zone, :int_zone], 0)
        print(tripPeriod, purp, 'SR2', " ", np.array(util['SR2']).min(), np.array(util['SR2']).max())

        #Shared ride 3 utility
        util['SR3'] = np.where(HOV3_TIME[tripPeriod][:int_zone, :int_zone]>0, c_ivt*HOV3_TIME[tripPeriod][:int_zone, :int_zone], 0)
        print(tripPeriod, purp, 'SR3', " ", np.array(util['SR3']).min(), np.array(util['SR3']).max())

        # Walk  utility
        util['WALK'] = np.where(walk_dist>0, (walk_dist<=1)* (c_walkTimeShort * np.minimum(walk_dist * 60 / walkSpeed, walkThresh * 60 / walkSpeed)) + \
                       (walk_dist>1)* (c_walkTimeLong * np.maximum(walk_dist * 60 / walkSpeed, walkThresh * 60 / walkSpeed)) ,0) 
        print(tripPeriod, purp, 'WALK', " ", np.array(util['WALK']).min(), np.array(util['WALK']).max())
        
        #Bike utility 
        util['BIKE'] =  np.where(bike_dist>0, (bike_dist<=6)*(c_bikeTimeShort* np.minimum(bike_dist*60/bikeSpeed, bikeThresh*60/bikeSpeed)) + \
                       (bike_dist>6)*(c_bikeTimeLong* np.maximum(bike_dist*60/bikeSpeed, bikeThresh*60/bikeSpeed)), 0)
        print(tripPeriod, purp, 'BIKE', " ", np.array(util['BIKE']).min(), np.array(util['BIKE']).max())
        
        #Walk transit Walk utility
        util['WLK_TRN_WLK'] =  np.where(WLK_TRN_WLK_TOTIVT[tripPeriod] > 0, 
                                    c_ivt*WLK_TRN_WLK_IVT_LOC[tripPeriod]/100 + \
                                    c_ivt_exp*WLK_TRN_WLK_IVT_EXP[tripPeriod]/100 + \
                                    c_ivt_lrt*WLK_TRN_WLK_IVT_LRT[tripPeriod]/100 + \
                                    c_ivt_ferry*WLK_TRN_WLK_IVT_FRY[tripPeriod]/100 + \
                                    c_ivt_hvy*WLK_TRN_WLK_IVT_HVY[tripPeriod]/100 + \
                                    c_ivt_com*WLK_TRN_WLK_IVT_COM[tripPeriod]/100 + \
                                    c_ivt_trn_crwd*WLK_TRN_WLK_CROWD[tripPeriod]/100 + \
                                    c_shortiWait*np.minimum(WLK_TRN_WLK_IWAIT[tripPeriod]/100,waitThresh) + \
                                    c_longiWait*np.maximum(WLK_TRN_WLK_IWAIT[tripPeriod]/100-waitThresh,0) + \
                                    c_xwait*WLK_TRN_WLK_XWAIT[tripPeriod]/100 + \
                                    c_xfers_wlk * np.maximum(WLK_TRN_WLK_BOARDS[tripPeriod]-1,0) + \
                                    c_waux*WLK_TRN_WLK_WAUX[tripPeriod]/100, 0)
        print(tripPeriod, purp, 'WLK_TRN_WLK', " ", np.array(util['WLK_TRN_WLK']).min(), np.array(util['WLK_TRN_WLK']).max())
        
        # Walk Transit PNR - Inbound utility
        util['WLK_TRN_PNR'] =  np.where(WLK_TRN_PNR_TOTIVT[tripPeriod] > 0,
                                c_ivt*WLK_TRN_PNR_TOTIVT[tripPeriod]/100 + \
                                (c_ivt_com-c_ivt)*WLK_TRN_PNR_IVT_COM[tripPeriod]/100 + \
                                c_ivt_trn_crwd*WLK_TRN_PNR_CROWD[tripPeriod]/100 + \
                                c_shortiWait*np.minimum(WLK_TRN_PNR_IWAIT[tripPeriod]/100,waitThresh) + \
                                c_longiWait*np.maximum(WLK_TRN_PNR_IWAIT[tripPeriod]/100-waitThresh,0) + \
                                c_xwait*WLK_TRN_PNR_XWAIT[tripPeriod]/100 + \
                                c_dtim*WLK_TRN_PNR_DTIM[tripPeriod]/100 + \
                                c_xfers_drv*np.maximum(WLK_TRN_PNR_BOARDS[tripPeriod]-1,0) + \
                                c_waux*WLK_TRN_PNR_WAUX[tripPeriod]/100 + \
                                c_dacc_ratio*(WLK_TRN_PNR_DDIST[tripPeriod]/100/SOV_DIST[tripPeriod][:int_zone, :int_zone]), 0)

        print(tripPeriod, purp, 'WLK_TRN_PNR', " ", np.array(util['WLK_TRN_PNR']).min(), np.array(util['WLK_TRN_PNR']).max())

        # PNR transit Walk - Outbound utility
        util['PNR_TRN_WLK'] = np.where(PNR_TRN_WLK_TOTIVT[tripPeriod]>0, 
                            c_ivt*PNR_TRN_WLK_TOTIVT[tripPeriod]/100 + \
                            (c_ivt_com-c_ivt)*PNR_TRN_WLK_IVT_COM[tripPeriod]/100 + \
                            c_ivt_trn_crwd*PNR_TRN_WLK_CROWD[tripPeriod]/100 + \
                            c_shortiWait*np.minimum(PNR_TRN_WLK_IWAIT[tripPeriod]/100,waitThresh) + \
                            c_longiWait*np.maximum(PNR_TRN_WLK_IWAIT[tripPeriod]/100-waitThresh,0) + \
                            c_xwait*PNR_TRN_WLK_XWAIT[tripPeriod]/100 + \
                            c_dtim*PNR_TRN_WLK_DTIM[tripPeriod]/100 + \
                            c_xfers_drv*np.maximum(PNR_TRN_WLK_BOARDS[tripPeriod]-1,0) + \
                            c_waux*PNR_TRN_WLK_WAUX[tripPeriod]/100 + \
                            c_dacc_ratio*(PNR_TRN_WLK_DDIST[tripPeriod]/100/SOV_DIST[tripPeriod][:int_zone, :int_zone]),0)
        print(tripPeriod, purp, 'PNR_TRN_WLK', " ", np.array(util['PNR_TRN_WLK']).min(), np.array(util['PNR_TRN_WLK']).max())

        # Walk Transit KNR - Inbound utility
        util['WLK_TRN_KNR'] = np.where(WLK_TRN_KNR_TOTIVT[tripPeriod]>0, 
                                c_ivt*WLK_TRN_KNR_TOTIVT[tripPeriod]/100 + \
                                (c_ivt_com-c_ivt)*WLK_TRN_KNR_IVT_COM[tripPeriod]/100 + \
                                c_ivt_trn_crwd*WLK_TRN_KNR_CROWD[tripPeriod]/100 + \
                                c_shortiWait*np.minimum(WLK_TRN_KNR_IWAIT[tripPeriod]/100,waitThresh) + \
                                c_longiWait*np.maximum(WLK_TRN_KNR_IWAIT[tripPeriod]/100-waitThresh,0) + \
                                c_xwait*WLK_TRN_KNR_XWAIT[tripPeriod]/100 + \
                                c_xfers_drv*np.maximum(WLK_TRN_KNR_BOARDS[tripPeriod]-1,0) + \
                                c_dtim*WLK_TRN_KNR_DTIM[tripPeriod]/100 + \
                                c_waux*WLK_TRN_KNR_WAUX[tripPeriod]/100 + \
                                c_dacc_ratio*(WLK_TRN_KNR_DDIST[tripPeriod]/100/SOV_DIST[tripPeriod][:int_zone, :int_zone]),0)
        
        print(tripPeriod, purp, 'WLK_TRN_KNR', " ", np.array(util['WLK_TRN_KNR']).min(), np.array(util['WLK_TRN_KNR']).max())

        # KNR Transit Walk - Outbound utility
        util['KNR_TRN_WLK'] = np.where(KNR_TRN_WLK_TOTIVT[tripPeriod]>0, 
                            c_ivt*KNR_TRN_WLK_TOTIVT[tripPeriod]/100 + \
                            (c_ivt_com-c_ivt)*KNR_TRN_WLK_IVT_COM[tripPeriod]/100 + \
                            c_ivt_trn_crwd*KNR_TRN_WLK_CROWD[tripPeriod]/100 + \
                            c_shortiWait*np.minimum(KNR_TRN_WLK_IWAIT[tripPeriod]/100,waitThresh) + \
                            c_longiWait*np.maximum(KNR_TRN_WLK_IWAIT[tripPeriod]/100-waitThresh,0) + \
                            c_xwait*KNR_TRN_WLK_XWAIT[tripPeriod]/100 + \
                            c_xfers_drv*np.maximum(KNR_TRN_WLK_BOARDS[tripPeriod]-1,0) + \
                            c_dtim*KNR_TRN_WLK_DTIM[tripPeriod]/100 + \
                            c_waux*KNR_TRN_WLK_WAUX[tripPeriod]/100 + \
                            c_dacc_ratio*(KNR_TRN_WLK_DDIST[tripPeriod]/100/SOV_DIST[tripPeriod][:int_zone, :int_zone]), 0)
        
        print(tripPeriod, purp, 'KNR_TRN_WLK', " ", np.array(util['KNR_TRN_WLK']).min(), np.array(util['KNR_TRN_WLK']).max())

        # taxi
        util['RIDEHAIL'] = np.where(HOV2_TIME[tripPeriod][:int_zone, :int_zone]>0, c_ivt*HOV2_TIME[tripPeriod][:int_zone, :int_zone]  + 
                                    c_ivt*1.5*taxi_wait_time, 0)
        print(tripPeriod, purp, 'RIDEHAIL', " ", np.array(util['RIDEHAIL']).min(), np.array(util['RIDEHAIL']).max())
          
        util.close()

Work
1 Work DA   -22000.0 -0.0025085662
1 Work SR2   -22000.0 -0.0025085662
1 Work SR3   -22000.0 -0.0025085662
1 Work WALK   -19.223921100079313 0.0
1 Work BIKE   -47.724191393766105 0.0
1 Work WLK_TRN_WLK   -53.895435 0.0
1 Work WLK_TRN_PNR   -26.306095 0.0
1 Work PNR_TRN_WLK   -38.615494 0.0
1 Work WLK_TRN_KNR   -26.217352 0.0
1 Work KNR_TRN_WLK   -38.18076 0.0
1 Work RIDEHAIL   -22000.3399 -0.10150856624916196
2 Work DA   -22000.0 -0.002519961
2 Work SR2   -22000.0 -0.002519961
2 Work SR3   -22000.0 -0.002519961
2 Work WALK   -19.223921100079313 0.0
2 Work BIKE   -47.724191393766105 0.0
2 Work WLK_TRN_WLK   -34.031075 0.0
2 Work WLK_TRN_PNR   -29.945293 0.0
2 Work PNR_TRN_WLK   -18.136559 0.0
2 Work WLK_TRN_KNR   -29.937073 0.0
2 Work KNR_TRN_WLK   -15.3798065 0.0
2 Work RIDEHAIL   -22000.3399 -0.10151996098086238
3 Work DA   -22000.0 -0.0025357152
3 Work SR2   -22000.0 -0.0025357152
3 Work SR3   -22000.0 -0.0025357152
3 Work WALK   -19.223921100079313 0.0
3 Work BIKE   -47.7241913

4 Escort WLK_TRN_PNR   -36.77291 0.0
4 Escort PNR_TRN_WLK   -23.803896 0.0
4 Escort WLK_TRN_KNR   -37.063282 0.0
4 Escort KNR_TRN_WLK   -22.824379 0.0
4 Escort RIDEHAIL   -27900.431055 -0.12874262431003153
5 Escort DA   -27900.0 -0.0031830622
5 Escort SR2   -27900.0 -0.0031830622
5 Escort SR3   -27900.0 -0.0031830622
5 Escort WALK   -24.379427213282405 0.0
5 Escort BIKE   -60.52295181300339 0.0
5 Escort WLK_TRN_WLK   -56.97457 0.0
5 Escort WLK_TRN_PNR   -40.56631 0.0
5 Escort PNR_TRN_WLK   -37.732277 0.0
5 Escort WLK_TRN_KNR   -37.44 0.0
5 Escort KNR_TRN_WLK   -34.017036 0.0
5 Escort RIDEHAIL   -27900.431055 -0.1287330621883273
Shopping
1 Shopping DA   -27900.0 -0.003181318
1 Shopping SR2   -27900.0 -0.003181318
1 Shopping SR3   -27900.0 -0.003181318
1 Shopping WALK   -24.379427213282405 0.0
1 Shopping BIKE   -60.52295181300339 0.0
1 Shopping WLK_TRN_WLK   -67.67271 0.0
1 Shopping WLK_TRN_PNR   -33.360912 0.0
1 Shopping PNR_TRN_WLK   -48.97146 0.0
1 Shopping WLK_TRN_KNR   -33.248367 0.

3 Social DA   -27900.0 -0.003215748
3 Social SR2   -27900.0 -0.003215748
3 Social SR3   -27900.0 -0.003215748
3 Social WALK   -24.379427213282405 0.0
3 Social BIKE   -60.52295181300339 0.0
3 Social WLK_TRN_WLK   -54.391727 0.0
3 Social WLK_TRN_PNR   -35.169624 0.0
3 Social PNR_TRN_WLK   -34.744858 0.0
3 Social WLK_TRN_KNR   -34.653847 0.0
3 Social KNR_TRN_WLK   -31.234478 0.0
3 Social RIDEHAIL   -27900.431055 -0.128765747885406
4 Social DA   -27900.0 -0.0031926243
4 Social SR2   -27900.0 -0.0031926243
4 Social SR3   -27900.0 -0.0031926243
4 Social WALK   -24.379427213282405 0.0
4 Social BIKE   -60.52295181300339 0.0
4 Social WLK_TRN_WLK   -37.937965 0.0
4 Social WLK_TRN_PNR   -36.77291 0.0
4 Social PNR_TRN_WLK   -23.803896 0.0
4 Social WLK_TRN_KNR   -37.063282 0.0
4 Social KNR_TRN_WLK   -22.824379 0.0
4 Social RIDEHAIL   -27900.431055 -0.12874262431003153
5 Social DA   -27900.0 -0.0031830622
5 Social SR2   -27900.0 -0.0031830622
5 Social SR3   -27900.0 -0.0031830622
5 Social WALK   -24

In [14]:
### END